# 📦 Step 1: Import Libraries

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Step 2: Generate Synthetic Exam Registration Data

In [2]:
np.random.seed(42)

n = 1000
student_ids = np.random.randint(100000, 999999, size=n)
exam_types = np.random.choice(['SAT', 'PSAT', 'AP'], size=n)
registration_status = np.random.choice(['Success', 'Failed'], size=n, p=[0.85, 0.15])
error_codes = [None if status == 'Success' else random.choice(['ERR01', 'ERR02', 'ERR03']) for status in registration_status]
timestamps = [datetime.now() - timedelta(days=np.random.randint(0, 30)) for _ in range(n)]

df = pd.DataFrame({
    'student_id': student_ids,
    'exam_type': exam_types,
    'registration_status': registration_status,
    'error_code': error_codes,
    'timestamp': timestamps
})

df.head()

,student_id,exam_type,registration_status,error_code,timestamp
0,221958,SAT,Failed,ERR03,2025-05-21 20:52:29.113092
1,771155,SAT,Failed,ERR03,2025-06-08 20:52:29.113165
2,231932,SAT,Success,None,2025-05-24 20:52:29.113176
3,465838,PSAT,Success,None,2025-05-16 20:52:29.113184
4,359178,SAT,Failed,ERR02,2025-06-10 20:52:29.113190


# Step 3: Simulate AWS Athena/Redshift using SQLite

In [3]:
import sqlite3

conn = sqlite3.connect(':memory:')
df.to_sql('exam_data', conn, index=False)

# Query 1: Count of registrations by status
query1 = """
SELECT registration_status, COUNT(*) as count
FROM exam_data
GROUP BY registration_status
"""
pd.read_sql_query(query1, conn)

,registration_status,count
0,Failed,134
1,Success,866


# Query 2: Root cause breakdown for failed registrations

In [4]:
query2 = """
SELECT error_code, COUNT(*) as error_count
FROM exam_data
WHERE registration_status = 'Failed'
GROUP BY error_code
ORDER BY error_count DESC
"""
pd.read_sql_query(query2, conn)

,error_code,error_count
0,ERR03,46
1,ERR02,46
2,ERR01,42


# Optional: Visualize with Plotly

In [5]:
!pip install plotly


In [6]:
import plotly.express as px

# Prepare data
error_summary = pd.read_sql_query(query2, conn)

# Plot
fig = px.bar(error_summary, x='error_code', y='error_count',
             title='Root Cause Analysis: Failed Registrations',
             labels={'error_code': 'Error Code', 'error_count': 'Number of Failures'})
fig.show()


## ✅ Summary

In this simulation, we demonstrated:
- How to use SQL to conduct root cause analysis on registration failures
- How AWS-style querying (Athena/Redshift) can be replicated using SQLite
- How to visualize operational problems using interactive charts
- How to structure workflows that resemble cloud-based pipelines (e.g., S3 + Athena)

This notebook is a strong example of how I’ve applied **AWS analytics tools**, **SQL expertise**, and **problem-solving skills**—exactly the kind of experience highlighted in my application to the **College Board’s Exam Config team**.
